In [1]:
import pandas as pd
import glob
import os

In [2]:
os.chdir('/home1/gkrtod35/ISF/TimeGAN/Origin_data')

In [3]:
df = pd.read_csv('merged_data_processed_seoul.csv', low_memory=False)
df.head()

,Idx,date,time,solar generation,일시,기온(°C),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),...,일조(hr),일사(MJ/m2),적설(cm),전운량(10분위),중하층운량(10분위),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,0,2014-01-01,0,0.0,2014-01-01 00:00,3.3,3.8,250.0,65.0,5.0,...,0.0,0.0,0.0,6.0,6.0,0.0,0.1,-0.2,0.0,1.5
1,0,2014-01-01,1,0.0,2014-01-01 01:00,2.6,2.3,250.0,66.0,4.9,...,0.0,0.0,0.0,0.0,0.0,-0.1,0.1,-0.2,0.1,1.5
2,0,2014-01-01,2,0.0,2014-01-01 02:00,1.7,1.7,250.0,67.0,4.6,...,0.0,0.0,0.0,0.0,0.0,-0.3,0.0,-0.2,0.0,1.5
3,0,2014-01-01,3,0.0,2014-01-01 03:00,1.4,1.4,250.0,60.0,4.1,...,0.0,0.0,0.0,0.0,0.0,-0.4,0.0,-0.2,0.1,1.5
4,0,2014-01-01,4,0.0,2014-01-01 04:00,0.9,2.8,270.0,59.0,3.8,...,0.0,0.0,0.0,0.0,0.0,-0.6,0.0,-0.2,0.0,1.5


In [9]:
# 
seq_len  = 8760   # 학습용: 1년치 (1h 단위 → 8760h)
pred_len =   24   # 테스트용: 1일치 (24h)

# 꼬리(tail)에서 잘라내기
#    – train_data: 마지막(pred_len)시간 바로 앞의 seq_len시간
#    – test_data : 마지막 pred_len시간
df = df[-(seq_len + pred_len) : ]  # shape (8760, C)

In [10]:
df

,Idx,date,time,solar generation,일시,기온(°C),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),...,일조(hr),일사(MJ/m2),적설(cm),전운량(10분위),중하층운량(10분위),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
78120,3255,2022-11-30,0,0,2022-11-30 00:00,-1.6,5.5,290.0,43.0,2.3,...,0.0,0.0,0.0,8.0,8.0,0.6,7.8,8.3,9.5,10.7
78121,3255,2022-11-30,1,0,2022-11-30 01:00,-2.4,4.8,290.0,48.0,2.5,...,0.0,0.0,0.0,2.0,2.0,0.1,7.5,7.9,9.2,10.6
78122,3255,2022-11-30,2,0,2022-11-30 02:00,-3.2,5.4,270.0,52.0,2.5,...,0.0,0.0,0.0,9.0,5.0,0.0,7.1,7.6,9.0,10.5
78123,3255,2022-11-30,3,0,2022-11-30 03:00,-4.1,6.5,290.0,53.0,2.4,...,0.0,0.0,0.0,0.0,0.0,-0.3,6.7,7.3,8.8,10.4
78124,3255,2022-11-30,4,0,2022-11-30 04:00,-4.7,4.3,320.0,56.0,2.4,...,0.0,0.0,0.0,0.0,0.0,-0.7,6.4,7.0,8.5,10.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86899,3620,2023-11-30,19,0.10472,2023-11-30 19:00,-2.9,3.6,250.0,52.0,2.6,...,0.0,0.0,0.0,0.0,0.0,-1.8,1.6,2.0,3.0,4.5
86900,3620,2023-11-30,20,0.10744,2023-11-30 20:00,-3.3,2.4,270.0,52.0,2.5,...,0.0,0.0,0.0,0.0,0.0,-2.0,1.6,1.9,2.9,4.4
86901,3620,2023-11-30,21,0.10752,2023-11-30 21:00,-3.8,1.9,270.0,55.0,2.5,...,0.0,0.0,0.0,0.0,0.0,-2.3,1.5,1.9,2.9,4.4
86902,3620,2023-11-30,22,0.03700000000000001,2023-11-30 22:00,-4.1,1.6,270.0,53.0,2.4,...,0.0,0.0,0.0,0.0,0.0,-2.9,1.5,1.9,2.9,4.4
